In [1]:
from dataloading2 import nanopore_datamodule
from data_utils.split_methods import get_kfold_split_func, get_default_split
import pickle

with open(f'2022_blacklist_positives','rb') as f:
    bl = pickle.load(f)
    
dm = nanopore_datamodule(
    pos_files='pos_2022', 
    neg_files='neg_2022', 
    verbose=0, 
    batch_size=16, 
    valid_limit=10000, 
    workers=1,
    split_method=get_kfold_split_func(total_k=5, current_k=0),
    window=4096,
    read_blacklist = bl
)
dm.setup()
train_loader=iter(dm.train_dataloader())

In [2]:
from rnamodif.data_utils.datamap import experiment_files
experiment_files.keys()

dict_keys(['5eu_2022', 'UNM_2022', '5eu_2020', 'UNM_2020', 'remdesivir_33', 'ac4C_10', '2-OmeATP_10', 'remdesivir_0', 's4U_33', 's4U_5', 'm5C_10', 'm5C_5', '2-OmeATP_0', 's4U_0', 'm6A_33', 'ac4C_33', '2-OmeATP_5', 'remdesivir_5', 's4U_10', 'm5C_0', 'm6A_5', 'm6A_0', 'ac4C_0', 'm6A_10', 'm5c_novoa', 'm6a_novoa_1', 'm6a_novoa_2', 'UNM_novoa_1', 'UNM_novoa_2', '5eu_nanoid_24h', '5eu_nanoid_1h', '5eu_nanoid_mix', '5eu_nanoid_neg', 'empty'])

In [4]:
for k,v in experiment_files.items():
    print(k, len(v))

5eu_2022 275
UNM_2022 520
5eu_2020 150
UNM_2020 203
remdesivir_33 5
ac4C_10 18
2-OmeATP_10 11
remdesivir_0 28
s4U_33 21
s4U_5 31
m5C_10 24
m5C_5 24
2-OmeATP_0 11
s4U_0 28
m6A_33 22
ac4C_33 15
2-OmeATP_5 15
remdesivir_5 1
s4U_10 25
m5C_0 20
m6A_5 28
m6A_0 30
ac4C_0 15
m6A_10 33
m5c_novoa 104
m6a_novoa_1 134374
m6a_novoa_2 638860
UNM_novoa_1 66736
UNM_novoa_2 846595
5eu_nanoid_24h 177088
5eu_nanoid_1h 114106
5eu_nanoid_mix 138119
5eu_nanoid_neg 216906
empty 0


In [8]:
experiment_files['5eu_2022']

[PosixPath('/home/jovyan/local_store/store/seq/ont/experiments/20220303_hsa_dRNA_HeLa_5EU_polyA_REL5_2/runs/20220303_1920_MN36569_FAR89595_b610d332/fast5/FAR89595_30596c4a_14.fast5'),
 PosixPath('/home/jovyan/local_store/store/seq/ont/experiments/20220303_hsa_dRNA_HeLa_5EU_polyA_REL5_2/runs/20220303_1920_MN36569_FAR89595_b610d332/fast5/FAR89595_30596c4a_193.fast5'),
 PosixPath('/home/jovyan/local_store/store/seq/ont/experiments/20220303_hsa_dRNA_HeLa_5EU_polyA_REL5_2/runs/20220303_1920_MN36569_FAR89595_b610d332/fast5/FAR89595_30596c4a_116.fast5'),
 PosixPath('/home/jovyan/local_store/store/seq/ont/experiments/20220303_hsa_dRNA_HeLa_5EU_polyA_REL5_2/runs/20220303_1920_MN36569_FAR89595_b610d332/fast5/FAR89595_30596c4a_254.fast5'),
 PosixPath('/home/jovyan/local_store/store/seq/ont/experiments/20220303_hsa_dRNA_HeLa_5EU_polyA_REL5_2/runs/20220303_1920_MN36569_FAR89595_b610d332/fast5/FAR89595_30596c4a_41.fast5'),
 PosixPath('/home/jovyan/local_store/store/seq/ont/experiments/20220303_hsa_d

In [1]:
# from rnamodif_src.data_utils.datamap import experiment_files, experiments_dict
from pathlib import Path

list(Path('/home/jovyan/local_store').iterdir())
# Path().resolve()

[PosixPath('/home/jovyan/local_store/RNA010220191_m5C_fast5.tar.gz.1'),
 PosixPath('/home/jovyan/local_store/.ipynb_checkpoints'),
 PosixPath('/home/jovyan/local_store/store'),
 PosixPath('/home/jovyan/local_store/RNA010220191_m5C_fast5')]

In [11]:
tot = 0
for _ in range(100):
    x, y = next(train_loader)
    tot+=sum(y)
    
tot/100

tensor([7.8000])

In [7]:
from rnamodif.data_utils.datamap import experiment_files
experiment_files.keys()

dict_keys(['pos_2022', 'neg_2022', 'pos_2020', 'neg_2020', 'remdesivir_33', 'ac4C_10', '2-OmeATP_10', 'remdesivir_0', 's4U_33', 's4U_5', 'm5C_10', 'm5C_5', '2-OmeATP_0', 's4U_0', 'm6A_33', 'ac4C_33', '2-OmeATP_5', 'remdesivir_5', 's4U_10', 'm5C_0', 'm6A_5', 'm6A_0', 'ac4C_0', 'm6A_10', 'm5c_novoa'])

In [8]:
import pickle
total= {}
for k in range(5):
    with open(f'mislabeled_results_2022_{k}of5','rb') as f:
        results = pickle.load(f)
        total.update(results)
        print(len(results))
        
len(total)

614436
611264
614487
615032
611560


3066779

In [10]:
for k,v in total.items():
    total[k] = sum(v)/len(v)

In [ ]:
pos_mis = [v for k,v in total.items() if int(k[-2]) == 1]
neg_mis = [v for k,v in total.items() if int(k[-2]) == 0]
thresh = 0.99
mis_p = sum([p >= thresh for p in pos_mis])/len(pos_mis)
mis_n = sum([p >= thresh for p in neg_mis])/len(neg_mis)
print(mis_p)
print(format(mis_n, 'f'))

0.7306490371922131
0.000047


1041589

In [29]:
pos_mis = [k[:-3] for k,v in total.items() if (int(k[-2]) == 1 and v>=0.99)]
# pos_mis
len(pos_mis)

761036

In [28]:
import numpy as np
len(np.unique(pos_mis))


761036

In [31]:
with open(f'2022_blacklist_positives','wb') as f:
    pickle.dump(pos_mis, f)

In [32]:
with open(f'2022_blacklist_positives','rb') as f:
    bl = pickle.load(f)

In [35]:
bl

['a3eb87cf-4af1-4c7c-8af1-fb8032f154bc',
 'b1dcf6f8-030f-4864-aea4-fc34c02699b6',
 '300d1150-dcef-4bac-b601-8c9b2ba913b8',
 '4293a94f-7257-4f2d-8884-7ba365678ed6',
 '75d2b3e0-b7aa-4cf4-8219-d6c79c18d542',
 'bf5c0ba5-b264-414e-be2e-60595130830f',
 'c5d05fc8-4d01-4843-a2b1-af296dd9609e',
 '6013ba7b-028f-4ba3-a0e3-8a83b4215035',
 '1fb5be21-04ed-4f06-a7d6-4065bb894890',
 'ba5095bd-6fc0-4ae4-904b-623e304441cc',
 '17eb3972-bba8-40c6-8bb4-b87208deefcb',
 '2b1d0252-7b4c-405d-8eaa-18975f85a2cc',
 '0bdd35e6-a398-488a-affb-9a468e7124a4',
 'f3a0c0e1-bf60-4443-9052-a299fcea9a06',
 '37edc7d0-91fc-4123-bf2b-8f66785c7cf8',
 'c33cc6ad-fad8-42ae-be4a-b4f68c3d320c',
 '030ac48c-50cd-4350-a751-1db2e624d29b',
 'ca019eea-8171-4968-adfd-2b44a5b418fb',
 'd98bf59d-1dde-4b8b-885f-08c6aa90ac23',
 '8a9e9a2a-44f7-40ff-97b7-c8f62b2f431e',
 '8d52942a-3c74-4bfb-858a-5cf5f6fb5614',
 '15b4d2a7-d1dd-40dd-8188-55437976a526',
 '3012540e-68e2-486b-aa66-d154ac860e76',
 '2c3dcf34-b00b-4e25-96d5-96b3737730f9',
 '2f8d31f7-a825-

In [3]:
from pathlib import Path
import pickle

results_path = Path('mislabelings')
positives = list((results_path/'pos1').iterdir())
negatives = list((results_path/'neg1').iterdir())
logs = positives+negatives

In [2]:
len(logs)

0

In [15]:
with open(logs[5179366],'rb') as f:
    ident = pickle.load(f)
    print(ident)

IndexError: list index out of range

In [1]:
import torch
print(torch.__version__)

1.12.1


In [3]:
from RODAN.basecall import load_model
from types import SimpleNamespace
import torch
#TODO fix module importing without hacking imports
#TODO vocab ATCG - but rna is AUCG
torchdict = torch.load('./RODAN/rna.torch', map_location="cpu")
origconfig = torchdict["config"]
d = origconfig
n = SimpleNamespace(**d)
args = {
    'debug':False, #False
    'arch':None,
}
print(n)
# print(type(origconfig))
model, device = load_model('./RODAN/rna.torch', config=n, args=SimpleNamespace(**args))
# model

namespace(activation='mish', amp=False, arch='[[-1, 256, 0, 3, 1, 1, 0], [-1, 256, 1, 10, 1, 1, 1], [-1, 256, 1, 10, 10, 1, 1], [-1, 320, 1, 10, 1, 1, 1], [-1, 384, 1, 15, 1, 1, 1], [-1, 448, 1, 20, 1, 1, 1], [-1, 512, 1, 25, 1, 1, 1], [-1, 512, 1, 30, 1, 1, 1], [-1, 512, 1, 35, 1, 1, 1], [-1, 512, 1, 40, 1, 1, 1], [-1, 512, 1, 45, 1, 1, 1], [-1, 512, 1, 50, 1, 1, 1], [-1, 768, 1, 55, 1, 1, 1], [-1, 768, 1, 60, 1, 1, 1], [-1, 768, 1, 65, 1, 1, 1], [-1, 768, 1, 70, 1, 1, 1], [-1, 768, 1, 75, 1, 1, 1], [-1, 768, 1, 80, 1, 1, 1], [-1, 768, 1, 85, 1, 1, 1], [-1, 768, 1, 90, 1, 1, 1], [-1, 768, 1, 95, 1, 1, 1], [-1, 768, 1, 100, 1, 1, 1]]\n', batchsize=30, dropout=0.1, epochs=30, gradclip=0, half=False, lr=0.002, name='1210', optimizer='ranger', orig={'name': '1210', 'seqlen': 4096, 'epochs': 30, 'optimizer': 'ranger', 'lr': 0.002, 'weightdecay': 0.01, 'batchsize': 30, 'dropout': 0.1, 'activation': 'mish', 'sqex_activation': 'mish', 'sqex_reduction': 32, 'trainfile': '/s/neptune/a/tmp/newrn

In [4]:
model(torch.rand(8,1,4096).to('cuda')).size()

torch.Size([420, 8, 5])

In [2]:
from rodan_pretrained import RodanPretrained
import torch
model = RodanPretrained().cuda()
model(torch.rand(4,1,4096).to('cuda'))

tensor([[0.3219],
        [0.3033],
        [0.3036],
        [0.3361]], device='cuda:0')

In [ ]:
from torchmetrics import ConfusionMatrix
import torch
target = torch.tensor([1, 1, 1,0, 0,0,0])
preds = torch.tensor([0, 1,0,1,0, 0,0])
confmat = ConfusionMatrix(num_classes=2, normalize='true')
confmat(preds, target)


In [ ]:
from datamap import experiment_files
len(list(experiment_files['pos_2022_fast'][0].iterdir()))

In [ ]:
from dataloading2 import nanopore_datamodule
dm = nanopore_datamodule(pos_files = 'pos_2022_fast', neg_files='pos_2022_fast', verbose=True)

In [ ]:
import pytorch_lightning as pl
from torch import nn
import torch
from torch.nn import functional as F
from torch.nn import RNN
import torchmetrics
from resnet1d.resnet1d import ResNet1D




In [ ]:
from transformers import BertModel, BertConfig
BertModel(BertConfig())

In [ ]:
from tst import Transformer
from transformer.src.benchmark import LSTM, BiGRU, ConvGru, FullyConv, FFN
import torch
# LSTM(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
# BiGRU(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
# ConvGru(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
# FullyConv(input_dim=1, hidden_dim=64, output_dim=1)
# FFN(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)

#TODO chunk=none??
#TODO attention size
leng = 1000
model = Transformer(d_input=1, d_model=1, d_output=1,input_length=1000, q=8,v=8,h=8, N=4, attention_size=50, chunk_mode=None, pe='regular', skip_embedding=True)
# print(model)
print(model(torch.rand(32,1000,1)).size())
print(model)

In [ ]:
class GenericModel(pl.LightningModule):
    def __init__(self, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate


        self.net = LSTM(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
        # BiGRU(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
        # ConvGru(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
        # FullyConv(input_dim=1, hidden_dim=64, output_dim=1)
        # FFN(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)

        self.acc = torchmetrics.Accuracy()

    def forward(self, x):
        out = self.net.forward(x)
        return out[:,-1,:]
  
      

    def configure_optimizers(self):
      # print("LEARNING RATE:",self.learning_rate)
      optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=0.01) #wd 0.01
      return optimizer

    def training_step(self, train_batch, batch_idx):
      x,y = train_batch
      output = self(x)
      loss = F.binary_cross_entropy_with_logits(output, y)
      self.log('train_loss', loss)
      acc =self.acc(output, y.int())
      self.log('train acc', acc)
      return loss

    def validation_step(self, val_batch, batch_idx):
      x,y = val_batch
      output = self(x)
      loss = F.binary_cross_entropy_with_logits(output, y)
      self.log('valid_loss', loss)
      acc = self.acc(output, y.int())
      self.log('valid acc', acc)

rnn = GenericModel(0.001)
rnn(torch.swapaxes(torch.rand(32,1,1000),-1,-2)).size()
